In [1]:
import pandas as pd
import time
# import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.by import By

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

In [47]:
options = webdriver.ChromeOptions()
options.binary_location = "C:\Program Files\Google\Chrome Beta\Application\chrome.exe"
driver = webdriver.Chrome(executable_path=r'C:\Users\a-gue\.wdm\drivers\chromedriver\win32\104.0.5112.20\chromedriver.exe', chrome_options=options)

c:\users\a-gue\onedrive\documentos\python_projects\panama_apt_pricing\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\a-gue\onedrive\documentos\python_projects\panama_apt_pricing\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [48]:
# page_url = "https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos?q=number.50"
page_url = "https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama?q=withcat.real-estate-for-sale-apartments-condos|number.50"

In [49]:
driver.get(page_url)

##################################################

In [5]:
# In case of coockies button
# from selenium.webdriver.common.by import By
# driver.find_element(By.XPATH, '//div[text()="ACCEPT"]').click()

# Get url for all the 262 pages


In [6]:
pages_num = 261
url_list = []
for i in range(pages_num):

    # _url_decomposed = "https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos." + str(i+1) + "?q=number.50"
    _url_decomposed = "https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama." + str(i+1) + "?q=withcat.real-estate-for-sale-apartments-condos|number.50"
    url_list.append(_url_decomposed)


In [7]:
url_list

['https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama.1?q=withcat.real-estate-for-sale-apartments-condos|number.50',
 'https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama.2?q=withcat.real-estate-for-sale-apartments-condos|number.50',
 'https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama.3?q=withcat.real-estate-for-sale-apartments-condos|number.50',
 'https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama.4?q=withcat.real-estate-for-sale-apartments-condos|number.50',
 'https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama.5?q=withcat.real-estate-for-sale-apartments-condos|number.50',
 'https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama.6?q=withcat.real-estate-for-sale-apartments-condos|number.50',
 'https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-cond

In [31]:
# get individual url
url_by_add_test = driver.find_elements(By.XPATH, "//div[@class='ann-box-details']/a")
# url_by_add[0].get_attribute("href")
url_by_add_test[0].get_attribute("href")

'https://www.encuentra24.com/panama-en/real-estate-preconstruction-new-property/dynasty-residences-en-bella-vista/21727414?q=withcat.real-estate-for-sale-apartments-condos|number.50&regionslug=prov-panama&list=categoryregion&catslug=real-estate-for-sale-apartments-condos'

# Get Individual Links below commented

In [36]:
# url_by_add = []
# # iterate over link pages
# for i in url_list:
#     # get url on driver
#     driver.get(i)
#     time.sleep(3)
#     # find all box details
#     box_details = driver.find_elements(By.XPATH, "//div[@class='ann-box-details']/a")

#     for j in box_details:
#         url_by_add.append(j.get_attribute("href"))

In [40]:
# df_by_add = pd.DataFrame(url_by_add, columns=["url_by_add"])
# df_by_add.to_csv("url_by_add.csv")

In [8]:
# get titles of adds
items_per_page = driver.find_elements(By.XPATH, "//div/a/div[@itemprop='name']") # list of all adds
for i in range(len(items_per_page)):
   print(items_per_page[i].text) # individual title per add

Coco Place en Coco del Mar, San Francisco
Panama Viejo Residences, en Panama Viejo
Bali en Costa del Mar - Costa del Este
espectacular Apartment for sale in Ph Aqualina Punta Pacífica
For sale 3 bedrooms, 2 bathrooms, San Francisco, balcony, Parque Omar
Se vende apartamento en Santa María, Ph La Maison
Se vende apartamento en Santa María, Ph The Ivy
Se vende apartamento en Avenida Balboa, Ph Marina Terrace
Ph Lexington Tower. Apartamento en Venta Villa de las Fuentes - El Dorado - TUMBA MUERTO
Sale of 77 m2 Apartment in Viridian Park Carrasquilla
Se vende apartamento en The Reserve en Santa Maria - 243.76 mts DL
Yoo Panama Avenida Balboa, Modelo A con vista al mar
VENDO APARTAMENTO, CONDADO DEL REY PH KINGS PARK 2 RECAMARAS + CBE J.H
APARTAMENTOS EN THE RESERVE Torre PINNACLE SANTA MARÍA
2 bedrooms apartment for sale located in Obarrio
Beautiful fully furnished apartment for sale in Yacht Club Tower
GRAND TOWER - PUNTA PACIFICA
EMPIRE - SANTA MARIA con opción a 4 recamaras
GRAND TOWER 

# Get prices per item

In [9]:
# get prices per item
price_per_item = driver.find_elements(By.XPATH, "//span[@class='ann-price']")
# # not print(price_per_page[-1].text)  # Get Not when no element is found use the option below
# for i in price_per_page:
#     print(i.text)
len(price_per_item)

50

In [10]:
price_per_item = driver.find_elements(By.XPATH, "//div/span[@itemprop='offers']/div[@itemprop='price']")
for i in price_per_page:
    print(i.text)
len(price_per_page)

NameError: name 'price_per_page' is not defined

# Get area per item

In [21]:
area_per_item = driver.find_elements(By.XPATH, "//div/ul/li/span[@class='value']")

In [47]:
area_per_item[8].text[-3:]

'ago'

In [102]:
# for j in range(len(area_per_item)):
    # if j%2 ==0:
        
    # print(area_per_item[j].text)

In [105]:
len(area_per_item)

147

# Get Location

In [113]:
loc_per_item = driver.find_elements(By.XPATH, "//div/span[@class='ann-info-item']")

In [115]:
loc_per_item[0].text

'Avenida Balboa'

# Get Promotor

In [8]:
promotor_per_item = driver.find_elements(By.XPATH, "//div/span[@class='company-name']")

In [10]:
promotor_per_item[0].text

'PHRE Group'

#######################################################################################

# Data Extraction


In [59]:
# TODO: CREATE REPOSITORY

In [58]:
data = []
add_title = []
add_price = []
add_area = []
add_time = []
add_room = []
add_loc = []
add_prom = []
# iterate over links
for url in url_by_add[:2]:
    print(url)
    # get url on driver
    driver.get(url)
    # sleep 10 seconds just in case
    time.sleep(5)
    
    
    # get add and append
    # adds = driver.find_element(By.XPATH, "//div/a/div[@itemprop='name']") # list of all adds
    # for i in adds:
    #     add_title.append(i.text)
        
#     # get price and append
#     prices_1 = driver.find_elements(By.XPATH, "//span[@class='ann-price']")
#     for i in prices_1:
#         if i.text:
#             add_price.append(i.text)
#     prices_2 = driver.find_elements(By.XPATH, "//div/span[@itemprop='offers']/div[@itemprop='price']")
#     for i in prices_2:
#         if i.text:
#             add_price.append(i.text)
            
#     # get area and append
#     area = driver.find_elements(By.XPATH, "//div/ul/li/span[@class='value']")
#     for i in area:
#         if i.text[-2:] == "m2":
#             add_area.append(i.text)
            
#         # get time while looping
#         elif i.text[-3:] == "ago":
#             add_time.append(i.text)
        
#         elif len(i.text) > 3:
#             add_time.append("NaN")
#         # get rooms while looping:
#         else:
#             add_room.append(i.text)
            
#     # get location and appen
#     location = driver.find_elements(By.XPATH, "//div/span[@class='ann-info-item']")
#     for i in location:
#         add_loc.append(i.text)
    
#     # get promotor and append
#     promotor = driver.find_elements(By.XPATH, "//div/span[@class='company-name']")
#     for i in promotor:
#         if i:
#             add_prom.append(i.text)
#         else:
#             add_prom.append("NaN")
    

https://www.encuentra24.com/panama-en/real-estate-preconstruction-new-property/bali-en-costa-del-mar-costa-del-este/20982810?q=withcat.real-estate-for-sale-apartments-condos|number.50&regionslug=prov-panama&list=categoryregion&catslug=real-estate-for-sale-apartments-condos


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/a/div[@itemprop='name']"}
  (Session info: chrome=104.0.5112.81)
Stacktrace:
Backtrace:
	Ordinal0 [0x00DDAF23+2207523]
	Ordinal0 [0x00D730E1+1781985]
	Ordinal0 [0x00C83FE8+802792]
	Ordinal0 [0x00CB2290+991888]
	Ordinal0 [0x00CB252B+992555]
	Ordinal0 [0x00CDF6E2+1177314]
	Ordinal0 [0x00CCD634+1103412]
	Ordinal0 [0x00CDD9F2+1169906]
	Ordinal0 [0x00CCD406+1102854]
	Ordinal0 [0x00CA75F0+947696]
	Ordinal0 [0x00CA84E6+951526]
	GetHandleVerifier [0x01082A12+2733954]
	GetHandleVerifier [0x01074E28+2677656]
	GetHandleVerifier [0x00E646EA+513114]
	GetHandleVerifier [0x00E63796+509190]
	Ordinal0 [0x00D79CEB+1809643]
	Ordinal0 [0x00D7E448+1827912]
	Ordinal0 [0x00D7E535+1828149]
	Ordinal0 [0x00D87960+1866080]
	BaseThreadInitThunk [0x762BFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77B17A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77B17A6E+238]


In [53]:
# driver.get(url_by_add[0])


# Add missing values

In [31]:
while len(add_title) != len(add_price):
    add_price.append("NaN")

In [32]:
while len(add_title) != len(add_area):
    add_area.append("NaN")
    
while len(add_title) != len(add_time):
    add_time.append("NaN")
    
while len(add_title) != len(add_room):
    add_room.append("NaN")
    
while len(add_title) != len(add_prom):
    add_prom.append("NaN")

In [33]:
df = pd.DataFrame({"add_title":add_title, "add_price": add_price, "add_area": add_area, "add_time": add_time, "add_room": add_room, "add_loc": add_loc, "add_prom": add_prom})

In [34]:
df.to_csv("data_panama_only.csv")